In [1]:
# Import
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 評価指標
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [3]:
# データセットをダウンロード
from keras.datasets import mnist
(X, y), (X_test, y_test) = mnist.load_data()

ModuleNotFoundError: No module named 'keras'

In [ ]:
# データチェック
print(X.shape) # (60000, 28, 28)
print(X.shape) # (10000, 28, 28)
print(X[0].dtype) # uint8

In [ ]:
# 平滑化
X_flat = X.reshape(-1, 784)
X_test_flat = X_test.reshape(-1, 784)
print(X_flat.shape)
print(X_test_flat.shape)

In [ ]:
# 型変換、正規化
X_flat = X_flat.astype(np.float)
X_test_flat = X_test_flat.astype(np.float)
X_flat /= 255
X_test_flat /= 255
print(X_flat.max()) # 1.0
print(X_flat.min()) # 0.0

In [ ]:
# 正しいラベル値のワンホット エンコーディング
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_one_hot = enc.fit_transform(y[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
print(y.shape) # (60000,)
print(y_one_hot.shape) # (60000, 10)
print(y_one_hot.dtype) # float64

In [ ]:
# トレーニング データと検証データに分割
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_flat, y_one_hot, test_size=0.2)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

In [ ]:
# Scratch Deep Neural Network (今のところ、3層 NN)
class ScratchDeepNeuralNetworkClassifier():
    """
    N層ニューラル ネットワーク分類器
    Parameters
    ----------
    self.sigma : ガウス分布の標準偏差
    self.lr : 学習率
    self.n_nodes1 : 1層目のノード数
    self.n_nodes2 : 2層目のノード数
    self.n_output : 出力層のノード数
    
    self.n_epoch : エポック数
    self.n_batch : バッチ数
    self.verbose : 学習プロセスの可視化
    Attributes
    ----------
    """
    def __init__(self, n_features, n_nodes1, n_nodes2, n_output,
                 sigma, n_epoch, n_batch, lr, verbose = False):
        # Parameters
        self.n_features = n_features
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.n_output = n_output
        self.sigma = sigma
        self.n_epoch = n_epoch
        self.n_batch = n_batch
        self.lr = lr
        self.verbose = verbose
        self.log_loss = np.zeros(self.n_epoch)
        
    def loss_function(self,y,yt):
        delta = 1e-7
        return -np.mean(yt*np.log(y+delta))
                
    def fit(self, X, y, X_val=False, y_val=False):
        """
        ニューラル ネットワーク分類器をトレーニングする

        Parameters
        ----------
        X : 次の形の ndarray、形状 (n_samples、n_features)
            訓練データの特徴
        y : 次の形の ndarray, shape (n_samples, )
            学習データの正解値
        X_val : 次の形の ndarray、形状 (n_samples、n_features)
            検証データの特徴
        y_val : 次の形の ndarray, shape (n_samples, )
            検証データの正しい値
        """
        
        optimizer1 = AdaGrad(self.lr)
        optimizer2 = AdaGrad(self.lr)
        optimizer3 = AdaGrad(self.lr)
        
        initializer1 = XavierInitializer()
        initializer2 = XavierInitializer()
        initializer3 = SimpleInitializer(self.sigma)
        
        self.FC1 = FC(self.n_features, self.n_nodes1, initializer1, optimizer1, Tanh())
        self.FC2 = FC(self.n_nodes1, self.n_nodes2, initializer2, optimizer2, Tanh())
        self.FC3 = FC(self.n_nodes2, self.n_output, initializer3, optimizer3, Softmax())
        
        for epoch in range(self.n_epoch):
            # ミニバッチ処理
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.n_batch)
            
            self.loss = 0
            for mini_X_train, mini_y_train in get_mini_batch:
                
                # フォワードプロパゲーション
                self.Z1 = self.FC1.forward(mini_X_train)
                self.Z2 = self.FC2.forward(self.Z1)
                self.Z3 = self.FC3.forward(self.Z2)    

                # バックプロパゲーション
                self.dA3 = (self.Z3 - mini_y_train)/self.n_batch
                self.dZ2 = self.FC3.backward(self.dA3)
                self.dZ1 = self.FC2.backward(self.dZ2)
                self.dZ0 = self.FC1.backward(self.dZ1)
                
                # 損失関数
                self.loss += self.loss_function(self.Z3,mini_y_train)
                
            self.log_loss[epoch] = self.loss/len(get_mini_batch)
            
    def predict(self, X):
        """
        ニューラル ネットワーク分類器を使用して推定

        Parameters
        ----------
        X : 次の形の ndarray、形状 (n_samples、n_features)
            サンプル

        Returns
        -------
            次の形の ndarray、形状 (n_samples, 1)
            推定結果
        """
        pred_Z1 = self.FC1.forward(X)
        pred_Z2 = self.FC2.forward(pred_Z1)
        return np.argmax(self.FC3.forward(pred_Z2),axis=1)

In [ ]:
# ミニバッチ処理クラス
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形の ndarray、形状 (n_samples、n_features)
        訓練データ
    y : 次の形の ndarray、形状 (n_samples, 1) 
        正しい値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyでの乱数のシード
    """
    def __init__(self, X, y, batch_size = 20, seed=None):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
        
    def __len__(self):
        return self._stop
    
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1] 
    
    def __iter__(self):
        self._counter = 0
        return self
    
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [ ]:
# 問題1:全結合層のクラス化

class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer, activation):
        
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.initializer = initializer
        self.optimizer = optimizer
        self.activation = activation
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.W = self.initializer.W(self.n_nodes1,self.n_nodes2)
        self.B = self.initializer.B(self.n_nodes2)
        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        self.X = X
        self.A = np.dot(self.X,self.W) + self.B
        
        return self.activation.forward(self.A)
    
    def backward(self, dZ):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
             後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
             前に流す勾配
        """
        dA = self.activation.backward(dZ)
        self.dB = np.mean(dA,axis=0)
        self.dW = np.dot(self.X.T,dA)/len(self.X)
        dZ = np.dot(dA,self.W.T)
        
        # 更新
        self = self.optimizer.update(self)
        
        return dZ

In [ ]:
# 問題2:初期化方法のクラス化

class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma
        
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W : 重み
        """
        return self.sigma * np.random.randn(n_nodes1, n_nodes2)
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B : バイアス
        """
        return np.zeros(n_nodes2)

In [ ]:
# 問題3:最適化手法のクラス化

class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
        
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        layer.W -= self.lr*layer.dW
        layer.B -= self.lr*layer.dB
        
        return layer


In [ ]:
# 問題4:活性化関数のクラス化

class Tanh():
    """
    Activation function : ハイパボリックタンジェント関数
    """
    def __init__(self):
        pass
        
    def forward(self,A):
        self.A = A
        self.Z = np.tanh(self.A)
        return self.Z
    
    def backward(self,dZ):
        return dZ*(1-self.Z**2)
    
class Sigmoid():
    """
    Activation function : シグモイド関数
    """
    def __init__(self):
        pass
        
    def forward(self,A):
        self.A = A
        self.Z = 1/(1+np.exp(-self.A))
        
        return self.Z
    
    def backward(self,dZ):
        return dZ*(1-self.Z)*self.Z
    
class Softmax():
    """
    Activation Function : ソフトマックス関数
    """
    def __init__(self):
        pass
        
    def forward(self,A):
        
        return np.exp(A-np.max(A))/np.sum(np.exp(A-np.max(A)),axis=1,keepdims=True)
    
    def backward(self,dZ):
        return dZ

In [ ]:
# 問題5:ReLUクラスの作成

class ReLU():
    """
    Activation function : ReLU関数
    """
    def __init__(self):
        pass
        
    def forward(self,A):
        self.A = A
        return np.maximum(self.A,0)
    
    def backward(self,dZ):
        
        return np.where(self.A>0,dZ,0)
    
a = np.array([-1,0,1,9,-1])
b = np.array([1,0,-1,9,-1])

print(np.where(a<0,0,b))

In [ ]:
# 問題6:重みの初期値

class XavierInitializer():
    """
    Xavier で重みを初期化する
    """
    def __init__(self):
        pass
        
    def W(self, n_nodes1, n_nodes2):
        """
        重みを初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W : 重み
        """
        return np.random.randn(n_nodes1, n_nodes2)/np.sqrt(n_nodes1)
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B : バイアス
        """
        return np.zeros(n_nodes2)
    
class HeInitializer():
    """
    He による重みの初期化
    """
    def __init__(self):
        pass
        
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W : 重み
        """
        return np.random.randn(n_nodes1, n_nodes2)*np.sqrt(2/n_nodes1)
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B : バイアス
        """
        return np.zeros(n_nodes2)

In [ ]:
# 問題7:最適化手法

class AdaGrad:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
        self.hW = 0
        self.hB = 0
        
    def update(self, layer):
        """
        レイヤーの重みとバイアスの更新
        Parameters
        ----------
        layer : 更新前のレイヤーのインスタンス
        """
        self.hW += layer.dW*layer.dW
        self.hB = layer.dB*layer.dB
    
        layer.W -= self.lr*layer.dW/(np.sqrt(self.hW) +1e-7)
        layer.B -= self.lr*layer.dB/(np.sqrt(self.hB) +1e-7)
        
        return layer


In [ ]:
# 問題8:クラスの完成

clf = ScratchDeepNeuralNetworkClassifier(n_epoch=5, n_features=784,
                                         n_nodes1=400, n_nodes2=200, n_output=10,
                                         sigma=0.01, n_batch=100,
                                         lr = 0.01, verbose = False)

clf.fit(X_train,y_train)
y_pred = clf.predict(X_valid)

print(y_pred)

accuracy = accuracy_score(np.argmax(y_valid,axis=1), y_pred)
print('accuracy:{:.3f}'.format(accuracy))

# 各エポックの損失関数を可視化する
fig = plt.subplots(figsize=(12,8))
plt.rcParams["font.size"] = 20

plt.title('LOSS')
plt.plot(clf.log_loss,'rs--');

In [ ]:
# 問題9:学習と推定

# スクラッチディープニューラルネットワーク (任意のレイヤーで実行できる場合)
class ScratchDNNClassifier():
    """
    N層ニューラルネットワーク分類器
    
    Parameters
    ----------
    self.n_epoch : エポック数
    self.n_batch : バッチ数
    self.verbose : 学習プロセスの可視化
    Attributes
    ----------
    """
    def __init__(self, NN, n_epoch, n_batch, verbose = False):
        # Parameters
        self.n_epoch = n_epoch
        self.n_batch = n_batch
        self.verbose = verbose
        self.log_loss = np.zeros(self.n_epoch)
        self.log_acc = np.zeros(self.n_epoch)
        self.NN = NN
        
    def loss_function(self,y,yt):
        delta = 1e-7
        return -np.mean(yt*np.log(y+delta))
    
    def accuracy(self,Z,Y):
        return accuracy_score(Y,Z)
                
    def fit(self, X, y, X_val=False, y_val=False):
        """
        ニューラル ネットワーク分類器をトレーニング

        Parameters
        ----------
        X : 次の形の ndarray、形状 (n_samples、n_features)
            訓練データの特徴
        y : 次の形の ndarray, shape (n_samples, )
            学習データの正解値
        X_val : 次の形の ndarray、形状 (n_samples、n_features)
            検証データの特徴
        y_val : 次の形の ndarray, shape (n_samples, )
            検証データの正しい値
        """
        for epoch in range(self.n_epoch):
            # ミニバッチ処理
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.n_batch)
            
            self.loss = 0
            for mini_X_train, mini_y_train in get_mini_batch:
                
                # フォワードプロパゲーション
                forward_data = mini_X_train
                for layer in range(len(self.NN)):
                    forward_data = self.NN[layer].forward(forward_data)
                # 予測値
                Z = forward_data
                
                # バックプロパゲーション
                backward_data = (Z - mini_y_train)/self.n_batch
                for layer in range(len(self.NN)-1,-1,-1):
                    backward_data = self.NN[layer].backward(backward_data)
                
                # 損失関数
                self.loss += self.loss_function(Z,mini_y_train)
                
            self.log_loss[epoch] = self.loss/len(get_mini_batch)
            self.log_acc[epoch] = self.accuracy(self.predict(X),np.argmax(y,axis=1))
            
    def predict(self, X):
        """
        ニューラルネットワーク分類器を使用して推定

        Parameters
        ----------
        X : 次の形の ndarray、形状 (n_samples、n_features)
            サンプル

        Returns
        -------
            次の形の ndarray、形状 (n_samples, 1)
            推定結果
        """
        pred_data = X
        
        for layer in range(len(self.NN)):
            pred_data = self.NN[layer].forward(pred_data)
            
        return np.argmax(pred_data,axis=1)


In [ ]:
NN2 = {0:FC(784, 400, HeInitializer(), AdaGrad(0.01), ReLU()),
       1:FC(400, 200, HeInitializer(), AdaGrad(0.01), ReLU()),
       2:FC(200, 10, SimpleInitializer(0.01), AdaGrad(0.01), Softmax()),
       }

In [ ]:
clf2 = ScratchDNNClassifier(NN=NN2,n_epoch=30,n_batch=20,verbose = False)

clf2.fit(X_train,y_train)
y_pred2 = clf2.predict(X_valid)

In [ ]:
print(y_pred2)

accuracy = accuracy_score(np.argmax(y_valid,axis=1), y_pred2)
print('accuracy:{:.3f}'.format(accuracy))

In [ ]:
# 各エポックの損失関数を可視化
plt.rcParams["font.size"] = 20
fig=plt.subplots(figsize=(16,6))
plt.subplot(1,2,1)
plt.title('LOSS')
plt.plot(clf2.log_loss,'bo--')

plt.subplot(1,2,2)
plt.title('ACC')
plt.plot(clf2.log_acc,'rs--');

In [ ]:
NN3 = {0:FC(784, 400, HeInitializer(), AdaGrad(0.01), ReLU()),
       1:FC(400, 600, HeInitializer(), AdaGrad(0.01), ReLU()),
       2:FC(600, 300, HeInitializer(), AdaGrad(0.01), ReLU()),
       3:FC(300, 10, SimpleInitializer(0.01), AdaGrad(0.01), Softmax()),
       }

In [ ]:
clf3 = ScratchDNNClassifier(NN=NN3,n_epoch=30,n_batch=20,verbose = False)

clf3.fit(X_train,y_train)
y_pred3 = clf3.predict(X_valid)

In [ ]:
print(y_pred3)

accuracy = accuracy_score(np.argmax(y_valid,axis=1), y_pred3)
print('accuracy:{:.3f}'.format(accuracy))

In [ ]:
# 各エポックの損失関数を可視化
plt.rcParams["font.size"] = 20
fig=plt.subplots(figsize=(16,6))
plt.subplot(1,2,1)
plt.title('LOSS')
plt.plot(clf3.log_loss,'bo--')

plt.subplot(1,2,2)
plt.title('ACC')
plt.plot(clf3.log_acc,'rs--');

In [ ]:
NN4 = {0:FC(784, 400, XavierInitializer(), AdaGrad(0.01), Tanh()),
       1:FC(400, 300, XavierInitializer(), AdaGrad(0.01), Sigmoid()),
       2:FC(300, 200, HeInitializer(), AdaGrad(0.01), ReLU()),
       3:FC(200, 100, HeInitializer(), AdaGrad(0.01), ReLU()),
       4:FC(100, 10, SimpleInitializer(0.01), AdaGrad(0.01), Softmax()),
       }

In [ ]:
clf4 = ScratchDNNClassifier(NN=NN4,n_epoch=30,n_batch=20,verbose = False)

clf4.fit(X_train,y_train)
y_pred4 = clf4.predict(X_valid)

In [ ]:
print(y_pred4)

accuracy = accuracy_score(np.argmax(y_valid,axis=1), y_pred4)
print('accuracy:{:.3f}'.format(accuracy))

In [ ]:
# 各エポックの損失関数を可視化
plt.rcParams["font.size"] = 20
fig=plt.subplots(figsize=(16,6))
plt.subplot(1,2,1)
plt.title('LOSS')
plt.plot(clf4.log_loss,'bo--')

plt.subplot(1,2,2)
plt.title('ACC')
plt.plot(clf4.log_acc,'rs--');

In [ ]:
NN5 = {0:FC(784, 400, XavierInitializer(), AdaGrad(0.01), Tanh()),
       1:FC(400, 300, HeInitializer(), AdaGrad(0.01), ReLU()),
       2:FC(300, 200, HeInitializer(), AdaGrad(0.01), ReLU()),
       3:FC(200, 100, HeInitializer(), AdaGrad(0.01), ReLU()),
       4:FC(100, 10, SimpleInitializer(0.01), AdaGrad(0.01), Softmax()),
       }

In [ ]:
clf5 = ScratchDNNClassifier(NN=NN5,n_epoch=30,n_batch=20,verbose = False)

clf5.fit(X_train,y_train)
y_pred5 = clf5.predict(X_valid)

In [ ]:
print(y_pred5)

accuracy = accuracy_score(np.argmax(y_valid,axis=1), y_pred5)
print('accuracy:{:.3f}'.format(accuracy))

In [ ]:
# 各エポックの損失関数を可視化
plt.rcParams["font.size"] = 20
fig=plt.subplots(figsize=(16,6))
plt.subplot(1,2,1)
plt.title('LOSS')
plt.plot(clf5.log_loss,'bo--')

plt.subplot(1,2,2)
plt.title('ACC')
plt.plot(clf5.log_acc,'rs--');

In [ ]:
# Adagradをインスタンス化するだけで十分か
class TestScratchDeepNeuralNetworkClassifier():
    """
    N層ニューラルネットワーク分類器
    Parameters
    ----------
    self.sigma : ガウス分布の標準偏差
    self.lr : 学習率
    self.n_nodes1 : 第1層のノード数
    self.n_nodes2 : 第2層のノード数
    self.n_output : 出力層のノード数
    
    self.n_epoch : エポック数
    self.n_batch : バッチ数
    self.verbose : 学習プロセスの可視化
    Attributes
    ----------
    """
    def __init__(self, n_features, n_nodes1, n_nodes2, n_output,
                 sigma, n_epoch, n_batch, lr, verbose = False):
        # Parameters
        self.n_features = n_features
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.n_output = n_output
        self.sigma = sigma
        self.n_epoch = n_epoch
        self.n_batch = n_batch
        self.lr = lr
        self.verbose = verbose
        self.log_loss = np.zeros(self.n_epoch)
        
    def loss_function(self,y,yt):
        delta = 1e-7
        return -np.mean(yt*np.log(y+delta))
                
    def fit(self, X, y, X_val=False, y_val=False):
        """
        ニューラル ネットワーク分類器をトレーニング

        Parameters
        ----------
        X : 次の形の ndarray、形状 (n_samples、n_features)
            訓練データの特徴
        y : 次の形の ndarray, shape (n_samples, )
            学習データの正解値
        X_val : 次の形の ndarray、形状 (n_samples、n_features)
            検証データの特徴
        y_val : 次の形の ndarray, shape (n_samples, )
            検証データの正しい値
        """
        
        optimizer = AdaGrad(self.lr)
    
        initializer1 = XavierInitializer()
        initializer2 = XavierInitializer()
        initializer3 = SimpleInitializer(self.sigma)
        
        self.FC1 = FC(self.n_features, self.n_nodes1, initializer1, optimizer, Tanh())
        self.FC2 = FC(self.n_nodes1, self.n_nodes2, initializer2, optimizer, Tanh())
        self.FC3 = FC(self.n_nodes2, self.n_output, initializer3, optimizer, Softmax())
        
        for epoch in range(self.n_epoch):
            # ミニバッチ処理
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.n_batch)
            
            self.loss = 0
            for mini_X_train, mini_y_train in get_mini_batch:
                
                # フォワードプロパゲーション
                self.Z1 = self.FC1.forward(mini_X_train)
                self.Z2 = self.FC2.forward(self.Z1)
                self.Z3 = self.FC3.forward(self.Z2)    
                
                # バックプロパゲーション
                self.dA3 = self.Z3 - mini_y_train
                self.dZ2 = self.FC3.backward(self.dA3)
                self.dZ1 = self.FC2.backward(self.dZ2)
                self.dZ0 = self.FC1.backward(self.dZ1)
                
                # 損失関数
                self.loss += self.loss_function(self.Z3,mini_y_train)
                
            self.log_loss[epoch] = self.loss/self.n_batch
            
    def predict(self, X):
        """
        ニューラル ネットワーク分類器を使用して推定

        Parameters
        ----------
        X : 次の形の ndarray、形状 (n_samples、n_features)
            サンプル

        Returns
        -------
            次の形の ndarray、形状 (n_samples, 1)
            推定結果
        """
        pred_Z1 = self.FC1.forward(X)
        pred_Z2 = self.FC2.forward(pred_Z1)
        return np.argmax(self.FC3.forward(pred_Z2),axis=1)

In [ ]:
tst = TestScratchDeepNeuralNetworkClassifier(n_epoch=2, n_features=784,
                                             n_nodes1=400, n_nodes2=200,
                                             n_output=10, sigma=0.01, n_batch=100,
                                             lr = 0.01, verbose = False)

tst.fit(X_train,y_train)
y_pred = tst.predict(X_valid)